In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', 'src')))

In [2]:
# Import necessary libraries
from training import SummarizationDataLoader, Summarizer

import torch
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer

In [3]:
# Define los callbacks
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='../checkpoints/',  # Ruta donde se guardarán los modelos
    filename='best-checkpoint',
    save_top_k=1,
    mode='min'
)

early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min'
)

In [ ]:
data_module = SummarizationDataLoader(batch_size=8)
model = Summarizer()

In [ ]:
# Inicializa el entrenador con los callbacks
trainer = Trainer(
    max_epochs=10,
    callbacks=[checkpoint_callback, early_stopping_callback],
    devices=1 if torch.cuda.is_available() else 1,
    accelerator="gpu" if torch.cuda.is_available() else "cpu"
)

# Entrena el modelo
trainer.fit(model, datamodule=data_module)

In [ ]:
# Para cargar el mejor modelo guardado
best_model_path = checkpoint_callback.best_model_path

# Cargar el modelo desde el checkpoint
model = Summarizer.load_from_checkpoint(best_model_path)

print(f"El mejor modelo se ha guardado en: {best_model_path}")

In [ ]:
# Asegúrate de que el modelo esté en modo de evaluación
model.eval()

# Ejemplo de uso del modelo para hacer predicciones
# Supongamos que tienes una función de preprocesamiento y un texto de entrada
def preprocess(text):
    # Implementa la lógica de preprocesamiento aquí
    return text

input_text = "Este es un texto de ejemplo para resumir."
preprocessed_text = preprocess(input_text)

# Convertir el texto preprocesado a un tensor
input_tensor = torch.tensor(preprocessed_text)

# Hacer una predicción
with torch.no_grad():
    summary = model(input_tensor)

print("Resumen:", summary)